In [9]:
from os import getenv
from pymongo import MongoClient
from pandas import DataFrame, Series, concat
from dotenv import load_dotenv

In [35]:
filepath = '/Users/jasongersing/PycharmProjects/fastApiProject/.env'
load_dotenv(filepath)
url = getenv('MONGO_URL')
database = "thruTheGrapevine"
collection_GB = "GrapeBuyers"
collection_GS = "GrapeSellers"

In [36]:
db_GB = MongoClient(url)[database][collection_GB]
db_GB.count_documents({})

100

In [37]:
db_GS = MongoClient(url)[database][collection_GS]
db_GS.count_documents({})

20

In [39]:
grape_buyers = DataFrame(db_GB.find(projection={"_id": False, "grapes_seeking": True, "volume_seeking": True}))
grape_buyers = grape_buyers.explode(column=["grapes_seeking", "volume_seeking"])
grape_buyers.rename(columns={"grapes_seeking": "grapes", "volume_seeking": "volume"}, inplace=True)
grape_buyers["side"] = "Buyer"
grape_buyers["volume"] = Series([int(num) for num in grape_buyers["volume"]])
grape_buyers = grape_buyers.groupby(["grapes", "side"]).agg({"volume": "sum"}).reset_index()
grape_buyers

,grapes,side,volume
0,Cabernet Sauvignon,Buyer,405
1,Chardonnay,Buyer,195
2,Granache,Buyer,435
3,Marsanne,Buyer,495
4,Merlot,Buyer,370
5,Mourvedre,Buyer,445
6,Nebbiolo,Buyer,255
7,Petit Verdot,Buyer,385
8,Pinot blanc,Buyer,200
9,Pinot gris,Buyer,320


In [40]:
grape_sellers = DataFrame(db_GS.find(projection={"_id": False, "grapes_selling": True, "volume_selling": True}))
grape_sellers = grape_sellers.explode(column=["grapes_selling", "volume_selling"])
grape_sellers.rename(columns={"grapes_selling": "grapes", "volume_selling": "volume"}, inplace=True)
grape_sellers["side"] = "Seller"
grape_sellers["volume"] = Series([int(num) for num in grape_sellers["volume"]])
grape_sellers = grape_sellers.groupby(["grapes", "side"]).agg({"volume": "sum"}).reset_index()
grape_sellers

,grapes,side,volume
0,Cabernet Sauvignon,Seller,50
1,Chardonnay,Seller,50
2,Granache,Seller,90
3,Merlot,Seller,100
4,Mourvedre,Seller,145
5,Nebbiolo,Seller,135
6,Petit Verdot,Seller,30
7,Pinot blanc,Seller,145
8,Pinot gris,Seller,125
9,Pinot noir,Seller,160


In [41]:
total = concat([grape_buyers, grape_sellers])
total

,grapes,side,volume
0,Cabernet Sauvignon,Buyer,405
1,Chardonnay,Buyer,195
2,Granache,Buyer,435
3,Marsanne,Buyer,495
4,Merlot,Buyer,370
5,Mourvedre,Buyer,445
6,Nebbiolo,Buyer,255
7,Petit Verdot,Buyer,385
8,Pinot blanc,Buyer,200
9,Pinot gris,Buyer,320


In [299]:
import requests
from altair import Chart

In [301]:
chart = requests.get('http://127.0.0.1:8000/graph/df-grapes-by-side').json()
Chart.from_dict(chart)

alt.Chart(...)